<a href="https://colab.research.google.com/github/joomm/CNN/blob/main/%EC%95%95%EC%B6%95%ED%95%B4%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

폴더삭제코드

In [ ]:
import shutil

def remove_folders_with_contents(path):
    for folder_name in os.listdir(path):
        folder_path = os.path.join(path, folder_name)
        try:
            shutil.rmtree(folder_path)
            print(f"Deleted folder and its contents: {folder_path}")
        except OSError:
            print(f"Failed to delete folder and its contents: {folder_path}")

# 디렉토리 경로 설정
directory_path = '/content/08d94f4f5a7b2f737e2f91d423ccf1f95a3e0c8a1fa1d21c3b9bf34c79f5b374'

# 폴더와 내용 삭제 함수 호출
remove_folders_with_contents(directory_path)

Failed to delete folder and its contents: /content/08d94f4f5a7b2f737e2f91d423ccf1f95a3e0c8a1fa1d21c3b9bf34c79f5b374/layer.tar
Failed to delete folder and its contents: /content/08d94f4f5a7b2f737e2f91d423ccf1f95a3e0c8a1fa1d21c3b9bf34c79f5b374/json
Failed to delete folder and its contents: /content/08d94f4f5a7b2f737e2f91d423ccf1f95a3e0c8a1fa1d21c3b9bf34c79f5b374/VERSION


압축파일풀기

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks

!unzip -qq "/content/drive/MyDrive/03.AI모델_new.zip"

/content/drive/MyDrive/Colab Notebooks


In [ ]:
import os
import tarfile

target_folder = '/content/drive/MyDrive/Colab Notebooks/file/3fba5a28a1528a32ce6b5f4cb217b16eb4239a484e5280c36b99cc22e08fd4c7'

with tarfile.open(os.path.join(target_folder, 'layer.tar'), 'r') as f:
  f.extractall(target_folder)


---

---







In [ ]:
opencv-python
matplotlib
pillow
tqdm
numpy
albumentations
scikit-learn
pandas

In [99]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [100]:
import sys
# 모듈 저장
with open('config.py', 'w') as file:
    file.write("""
class config():
    path_prefix = '/content/drive/MyDrive/cnn/root'
    path_data = 'dataset'

    dict_label = {'11': {'00':0, '18': 1, '19': 2}}

    # general args
    gpu = 0
    train_size = (512, 512)
    train_model = 'resnet50' #'ENet'
    pnum = 77
    crop_name = '11'
    num_classes = 3

    # testing args
    test_model_path = '/content/drive/MyDrive/cnn/root/weights/sample.pt'
""")

# config 모듈이 현재 sys.modules에 있는지 확인
if 'config' in sys.modules:
    # 모듈이 로드되어 있다면 모듈을 다시 로드
    import importlib
    import config
    importlib.reload(config)

    # config 모듈 내용 확인
    print(config.__dict__)

else:
    # 모듈이 로드되어 있지 않다면 import 문을 통해 로드
    import config

{'__name__': 'config', '__doc__': None, '__package__': '', '__loader__': <_frozen_importlib_external.SourceFileLoader object at 0x7acdb8c82ad0>, '__spec__': ModuleSpec(name='config', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7acdb8c82ad0>, origin='/content/config.py'), '__file__': '/content/config.py', '__cached__': '/content/__pycache__/config.cpython-310.pyc', '__builtins__': {'__name__': 'builtins', '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.", '__package__': '', '__loader__': <class '_frozen_importlib.BuiltinImporter'>, '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in'), '__build_class__': <built-in function __build_class__>, '__import__': <built-in function __import__>, 'abs': <built-in function abs>, 'all': <built-in function all>, 'any': <built-in function any>, 'ascii': <built-in function ascii>, 'bin'

In [101]:
import albumentations

from config import config as cfg

test_aug = albumentations.Compose([albumentations.Normalize(mean=[0.485, 0.456, 0.406],
                                                             std=[0.229, 0.224, 0.225],
                                                             max_pixel_value=255.0,
                                                             p=1.0)
                                   ],
                                   p=1.0)

In [102]:
import torch.nn as nn
import torchvision.models as models

dict_backbone = {'resnet50' : models.resnet50}

def get_model(model_name='resnet50', num_classes=3, pretrained=False): # use pretrained backbone
    assert model_name in dict_backbone.keys()

    network = dict_backbone[model_name](pretrained=pretrained)
    network.fc = nn.Linear(network.fc.in_features, num_classes)

    return network

In [103]:
import os
import numpy as np
import datetime

import torch

def make_data(path, phase='train'):
    path_images = []

    for img_name in os.listdir(f'{path}/{phase}'):
        if img_name.endswith(('.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG')):
            path_images.append(f'{path}/{phase}/{img_name}')

    return np.stack(path_images)

def cnt_correct(y_true, y_pred):
    top_N, top_class = y_pred.topk(1, dim=-1)
    equals = top_class == y_true.view(*top_class.shape)
    return torch.sum(equals.type(torch.FloatTensor)).item()

def log_with_timestamp(msg):
    print(f'{datetime.datetime.today()} : {msg}')

def bash_cmd(cmd):
    print()
    _ = os.system(cmd)
    print()

In [104]:
import cv2
import numpy as np
import json

import torch
from torch.utils.data import Dataset

import transforms
from config import config as cfg

class CustomDataSet(Dataset):
    def __init__(self, path_images, transform, test=False):
        self.path_images = path_images
        self.transform = transform
        self.test = test

    def __getitem__(self, index):
        image = cv2.imread(self.path_images[index])[:,:,::-1]

        with open(f'{self.path_images[index]}.json') as json_file:
            json_decoded = json.load(json_file)
        label = cfg.dict_label[f'{int(json_decoded["annotations"]["crop"]):02d}'][f'{int(json_decoded["annotations"]["disease"]):02d}']

        resized_image = cv2.resize(image, cfg.train_size, interpolation = cv2.INTER_LANCZOS4) # INTER_AREA
        augmented_image = self.transform(image=resized_image)['image']

        if not self.test:
            return torch.tensor(augmented_image.transpose(2,0,1)), torch.tensor(label).long()
        else:
            return torch.tensor(augmented_image.transpose(2,0,1)), torch.tensor(label).long(), self.path_images[index]

    def __len__(self):
        return len(self.path_images)

In [105]:
import os
import numpy as np
from tqdm import tqdm
import pandas as pd

import torch
from torch.utils.data import DataLoader

import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel

import transforms
import datasets
import utils
from models import get_model
from config import config as cfg

from sklearn.metrics import confusion_matrix, classification_report

pnum = cfg.pnum
crop_name = cfg.crop_name

is_cuda = torch.cuda.is_available()

os.environ['CUDA_VISIBLE_DEVICES'] = str(cfg.gpu)

path_images_test = utils.make_data(f'{cfg.path_prefix}/{cfg.path_data}/{cfg.crop_name}', '1')

def test(model, valid_loader, criterion):
    model.eval()

    list_true = []
    list_pred = []

    loss_sum = 0
    correct_sum = 0

    log_every = len(valid_loader)//5

    len_data = 0

    utils.log_with_timestamp(f'Test ... 0%')

    for i, (image, label, name) in enumerate(valid_loader):
        if is_cuda and cfg.gpu is not None:
            image = image.cuda(cfg.gpu, non_blocking=True)
            label = label.cuda(cfg.gpu, non_blocking=True)

        pred = model(image)

        loss = criterion(pred, label)
        loss_sum += loss.item()

        len_data += image.shape[0]
        correct_sum += utils.cnt_correct(label, pred)

        list_true.append(label.detach().cpu().numpy().flatten())
        list_pred.append(pred.detach().cpu().numpy().argmax(-1).flatten())

        if (i+1)%log_every == 0:
            utils.log_with_timestamp(f'Test ... {((i+1)//log_every)*20}%')

    return loss_sum/len_data, correct_sum/len_data, np.concatenate(list_true, axis=0), np.concatenate(list_pred, axis=0)


def main():
    print('========= Test Environment =========\n')
    _ = utils.bash_cmd('cat /proc/cpuinfo | grep "model name" | uniq')
    _ = utils.bash_cmd('nvidia-smi')
    _ = utils.bash_cmd('free -h')
    _ = utils.bash_cmd('df /')
    _ = utils.bash_cmd('cat /etc/os-release')

    print('\nPyTorch version: ', torch.__version__, '\n')

    utils.log_with_timestamp('Model loading ... ')
    model = get_model(cfg.train_model, cfg.num_classes)
    ckpt = torch.load(cfg.test_model_path, map_location='cpu')
    model.load_state_dict(ckpt['model_state_dict'])

    utils.log_with_timestamp('Model compiling ... ')
    criterion = torch.nn.CrossEntropyLoss()

    if cfg.gpu is not None and is_cuda:
        utils.log_with_timestamp(f"Use GPU: {cfg.gpu} for testing")
        torch.cuda.set_device(cfg.gpu)
        model = model.cuda(cfg.gpu)
    else:
        utils.log_with_timestamp('Use CPU, this will be slow')

    test_loader = DataLoader(datasets.CustomDataSet(path_images_test, transforms.test_aug, test=True), batch_size=1, shuffle=False, num_workers=0)

    print('\n===== Start testing =====\n')
    test_loss, test_acc, list_true, list_pred = test(model, test_loader, criterion)

    print('\n===== Test result =====\n')
    print(classification_report(list_true, list_pred, digits=5))
    print('\n========= Confusion Matrix =========\n')
    for rr in confusion_matrix(list_true, list_pred):
        print(rr)

    print('\nLog csv ... ')
    df = pd.DataFrame({'image_name': path_images_test,
                       'true': list_true,
                       'pred': list_pred})

    df.to_csv(f'{pnum}_{crop_name}_log_each_sample.csv')

    print('\n===== Finished testing =====')

    _ = utils.bash_cmd('sh rm_cache.sh')

if __name__ == '__main__':
    main()

========= Test Environment =========












PyTorch version:  2.1.0+cu118 

2023-12-05 10:52:26.648211 : Model loading ... 
2023-12-05 10:52:28.487380 : Model compiling ... 
2023-12-05 10:52:28.488539 : Use GPU: 0 for testing

===== Start testing =====

2023-12-05 10:52:28.584344 : Test ... 0%


KeyError: ignored